In [1]:
%matplotlib inline

import numpy as np
import itertools
import random
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook

import torch
import torch.nn as nn
import torch.optim as optim

import sys
sys.path.append('/Users/andrew/Desktop/sudoku/src/sudoku')

from board import Board
from solutions import Solutions
import utils

In [2]:
# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)
torch.set_default_tensor_type('torch.DoubleTensor')

In [3]:
solutions = Solutions('/Users/andrew/Desktop/sudoku/data/solutions5.txt')
puzzles = solutions.get_puzzles_by_hints()
for hints in sorted(puzzles):
    print(hints, len(puzzles[hints]))

4 64
5 357
6 883
7 1584
8 2384
9 3309
10 4149
11 4754
12 4841
13 3741
14 1391
15 192
16 12


In [4]:
puzzles[4][0]

array([[1, 0, 0, 0],
       [0, 4, 0, 0],
       [0, 0, 0, 0],
       [2, 0, 0, 1]], dtype=int8)

In [15]:
def is_in_row(board, row, digit):
    return digit in board[row]

def is_in_col(board, col, digit):
    return digit in board[:,col]

def is_in_box(board, box, digit):
    return digit in board.get_box_by_index(box).box

def vectorize_cell(board, x, y):
    vector = torch.zeros(board.max_digit)
    if board[x][y] != 0:
        vector[board[x][y]-1] = 1
    return vector

In [17]:
max_digit = 4
model = Linear(max_digit)
model.double()

def closure():
    optimizer.zero_grad()
    prediction = model(X)
    total_cells = Y.shape[0]*Y.shape[1]
    flattened_prediction = prediction.reshape(total_cells, max_digit)
    flattened_Y = Y.reshape(total_cells)
    loss = nn.functional.nll_loss(flattened_prediction, flattened_Y)
    loss.backward()
    return loss

tensor([0., 0., 0., 1.])

In [44]:
class Query():
    ROW = "ROW"
    COLUMN = "COLUMN"
    BOX = "BOX"
    
    def __init__(self, board: Board, digit: int, exists: bool, house_type: int, house_index: int):
        assert house_type == Query.ROW or house_type == Query.COLUMN or house_type == Query.BOX
        assert 0 < digit <= board.max_digit
        assert 0 <= house_index < board.max_digit
        
        self.board = board
        self.digit = digit
        self.exists = exists
        self.house_type = house_type
        self.house_index = house_index
        
    def vectorize(self):
        exists = torch.zeros(2)
        digit = torch.zeros(self.board.max_digit)
        house_type = torch.zeros(3)
        house_index = torch.zeros(self.board.max_digit)
        
        exists[int(self.exists)] = 1
        digit[self.digit] = 1
        if self.house_type == Query.ROW:
            house_type[0] = 1
        elif self.house_type == Query.COLUMN:
            house_type[1] = 1
        else:
            house_type[2] = 1
        house_index[self.house_index] = 1
        
        return torch.cat([exists, digit, house_type, house_index])
    
    def answer(self):
        if self.house_type is Query.ROW:
            return is_in_row(self.board, self.house_index, self.digit)
        if self.house_type is Query.COLUMN:
            return is_in_col(self.board, self.house_index, self.digit)
        else:
            return is_in_box(self.board, self.house_index, self.digit)
    
    def relevant_cells(self):
        if self.house_type == Query.ROW:
            return [(self.house_index, i) for i in range(self.board.max_digit)]
        if self.house_type == Query.COLUMN:
            return [(i, self.house_index) for i in range(self.board.max_digit)]
        return self.board.get_box_by_index(self.house_index).get_coordinates()

In [45]:
puzzles[4][0]

array([[1, 0, 0, 0],
       [0, 4, 0, 0],
       [0, 0, 0, 0],
       [2, 0, 0, 1]], dtype=int8)

In [49]:
q = Query(puzzles[4][0], 1, True, Query.COLUMN, 0)
q.relevant_cells()

[(0, 0), (1, 0), (2, 0), (3, 0)]

In [24]:
class Response():
    MAYBE = "MAYBE"
    TRUE = "TRUE"
    FALSE = "FALSE"
    
    def __init__(self, x_dist, y_dist, answer_dist):
        self.x = torch.argmax(x_dist)
        self.y = torch.argmax(y_dist)
        
        answer_index = torch.argmax(answer_dist)
        if answer_index == 0:
            self.answer = Response.TRUE
        elif answer_index == 1:
            self.answer = Response.FALSE
        elif answer_index == 2:
            self.answer = Response.MAYBE
        assert len(answer_dist) == 2
            
    @staticmethod
    def answer_vector(answer):
        if answer == Response.TRUE:
            return torch.tensor([1, 0, 0])
        if answer == Response.FALSE:
            return torch.tensor([0, 1, 0])
        if answer == Response.MAYBE:
            return torch.tensor([0, 0, 1])
        assert answer in (Response.TRUE, Response.FALSE, Response.MAYBE)

In [ ]:
class Model1(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_sizes):
        super(Model1, self).__init__()
        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        self.output_sizes = output_sizes
        
        self.lstm = nn.LSTMCell(input_size, hidden_layer_size)
        self.outputs = nn.ModuleList() 
        for output_size in output_sizes:
            self.outputs.append(nn.Linear(hidden_layer_size, output_size))
        self.softmax = nn.Softmax()
        
    def forward(self, query_vector, board):
        h_t = torch.zeros(1, self.hidden_layer_size, dtype=torch.double)
        c_t = torch.zeros(1, self.hidden_layer_size, dtype=torch.double)
        
        relevant_seen = {}
        last_answer = answer_vector(Response.MAYBE)
        cell_vector = torch.zeros(board.max_digit)
        input = torch.cat([query_vector, cell_vector])
        
        while last_answer != Response.MAYBE and relevant_seen < 

In [6]:
class Model1(nn.Module):
    def __init__(self, query_dim, task1_dim, task2_dim, hidden_layer_size):
        super(Model1, self).__init__()
        self.input_dim = input_dim
        self.pointer_dim = pointer_dim
        self.task1_dim = task1_dim
        self.task2_dim = task2_dim
        self.hidden_layer_size = hidden_layer_size
        
        self.lstm = nn.LSTMCell(input_dim, hidden_layer_size)
        self.lin_pointer = nn.Linear(hidden_layer_size, pointer_dim)
        self.lin_task1 = nn.Linear(hidden_layer_size, task1_dim)
        self.lin_task2 = nn.Linear(hidden_layer_size, task2_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        pointer_outputs, task1_outputs, task2_outputs = [], [], []
        h_t = torch.zeros(1, self.hidden_layer_size, dtype=torch.double)
        c_t = torch.zeros(1, self.hidden_layer_size, dtype=torch.double)
        
        for x_t in x.reshape(len(x), 1, self.input_dim):
            h_t, c_t = self.lstm(x_t, (h_t, c_t))
            pointer_outputs.append(self.softmax(self.lin_pointer(h_t)))
            task1_outputs.append(self.softmax(self.lin_task1(h_t)))
            task2_outputs.append(self.softmax(self.lin_task2(h_t)))
        
        return torch.stack(pointer_outputs).reshape(len(x), self.pointer_dim), \
                torch.stack(task1_outputs).reshape(len(x), self.task1_dim), \
                torch.stack(task2_outputs).reshape(len(x), self.task2_dim)

True

In [7]:
is_in_col(puzzles[4][0], 0, 4)

False

In [8]:
4 in puzzles[4][0].get_box(0, 0).box

True

In [14]:
is_in_box(puzzles[4][0], 3, 1)

True

In [25]:
torch.cat([torch.tensor([1,2]), torch.tensor([3, 4])])

tensor([1, 2, 3, 4])